In [4]:
from glob import glob
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [57]:
r = requests.get('https://c.cari.com.my/')

In [58]:
soup = BeautifulSoup(r._content, "lxml")

In [61]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a]
a = [a_ for a_ in a if a_ and a_.startswith('portal.php?mod=list&catid=')]
a = ['https://c.cari.com.my/' + a_ for a_ in a]
a = sorted(list(set(a)))

In [76]:
import time

def get_url(url):
    while True:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "lxml")
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)
    try:
        span = soup.find_all('span')
        max_page = [s.get('title') for s in span if '页' in s.get('title', '') and '共' in s.get('title', '')][0]
        max_page = int(max_page.split()[1])
    except:
        max_page = -1
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if a_.startswith('portal.php?mod=view&aid=')]
    return url, a, max_page

In [64]:
# get_url(a[0])

In [65]:
r = []
for a_ in tqdm(a):
    r.append(get_url(a_))

100%|███████████████████████████████████████████| 28/28 [00:10<00:00,  2.68it/s]


In [66]:
with open('portals.json', 'w') as fopen:
    json.dump(r, fopen)

In [67]:
urls = []
for r_ in r:
    url = r_[0]
    for i in range(1, r_[2] + 1, 1):
        urls.append(url + f'&page={i}')
        
urls = sorted(list(set(urls)))
len(urls)

1474

In [68]:
def get_href(url):
    while True:
        try:
            r = requests.get(url, timeout = 10.0)
            break
        except:
            time.sleep(1.0)
    return r._content.decode()

In [69]:
!rm -rf pages
!mkdir pages

In [70]:
urls[:10]

['https://c.cari.com.my/portal.php?mod=list&catid=201&page=1',
 'https://c.cari.com.my/portal.php?mod=list&catid=201&page=2',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=1',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=10',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=11',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=12',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=13',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=14',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=15',
 'https://c.cari.com.my/portal.php?mod=list&catid=219&page=16']

In [77]:
import os

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
    
    filtered = []
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            filtered.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

100%|█████████████████████████████████████████| 148/148 [01:54<00:00,  1.29it/s]


In [78]:
def parse(content):
    soup = BeautifulSoup(content, "lxml")
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if a_.startswith('portal.php?mod=view&aid=')]
    return a

In [79]:
parse(filtered[0])

['portal.php?mod=view&aid=216754',
 'portal.php?mod=view&aid=216754',
 'portal.php?mod=view&aid=216755',
 'portal.php?mod=view&aid=216755',
 'portal.php?mod=view&aid=216844',
 'portal.php?mod=view&aid=216844',
 'portal.php?mod=view&aid=216772',
 'portal.php?mod=view&aid=216772',
 'portal.php?mod=view&aid=216763',
 'portal.php?mod=view&aid=216763',
 'portal.php?mod=view&aid=216719',
 'portal.php?mod=view&aid=216719',
 'portal.php?mod=view&aid=216741',
 'portal.php?mod=view&aid=216741',
 'portal.php?mod=view&aid=216740',
 'portal.php?mod=view&aid=216740',
 'portal.php?mod=view&aid=216733',
 'portal.php?mod=view&aid=216733',
 'portal.php?mod=view&aid=216720',
 'portal.php?mod=view&aid=216720',
 'portal.php?mod=view&aid=216731',
 'portal.php?mod=view&aid=216731',
 'portal.php?mod=view&aid=216730',
 'portal.php?mod=view&aid=216730',
 'portal.php?mod=view&aid=216729',
 'portal.php?mod=view&aid=216729',
 'portal.php?mod=view&aid=216718',
 'portal.php?mod=view&aid=216718',
 'portal.php?mod=vie

In [80]:
parse(filtered[1])

['portal.php?mod=view&aid=215909',
 'portal.php?mod=view&aid=215909',
 'portal.php?mod=view&aid=215911',
 'portal.php?mod=view&aid=215911',
 'portal.php?mod=view&aid=215910',
 'portal.php?mod=view&aid=215910',
 'portal.php?mod=view&aid=215907',
 'portal.php?mod=view&aid=215907',
 'portal.php?mod=view&aid=215902',
 'portal.php?mod=view&aid=215902',
 'portal.php?mod=view&aid=215901',
 'portal.php?mod=view&aid=215901',
 'portal.php?mod=view&aid=215900',
 'portal.php?mod=view&aid=215900',
 'portal.php?mod=view&aid=215899',
 'portal.php?mod=view&aid=215899',
 'portal.php?mod=view&aid=215898',
 'portal.php?mod=view&aid=215898',
 'portal.php?mod=view&aid=215871',
 'portal.php?mod=view&aid=215871',
 'portal.php?mod=view&aid=215872',
 'portal.php?mod=view&aid=215872',
 'portal.php?mod=view&aid=215865',
 'portal.php?mod=view&aid=215865',
 'portal.php?mod=view&aid=215862',
 'portal.php?mod=view&aid=215862',
 'portal.php?mod=view&aid=215861',
 'portal.php?mod=view&aid=215861',
 'portal.php?mod=vie